# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\List of Store Opening (PowerBI) 4.xlsx", sheet_name = 'PowerBI')
df.head(10)

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Jr Mx Centris Eton,Eton Centris Mall,For Opening,2024-05-16,MK5LZTE,NCR North,Sander Maghirang,MemoXpress
1,Jr Mx Mplace South Triangle Quezon City,Standalone,For Opening,2024-05-16,MORXEFK,NCR North,Sander Maghirang,MemoXpress
2,Jr Mx Sun Residences Quezon City,Standalone,For Opening,2024-05-16,MQDX72I,NCR North,Sander Maghirang,MemoXpress
3,Jr Mx Xentro Mall Vigan,Xentro Mall Vigan,For Opening,2024-05-16,MSVRCTU,North Luzon 2,Rachelle Belmonte,MemoXpress
4,Mx Robinsons Tuguegarao,Robinsons Tuguegarao,For Opening,2024-05-16,MZPBVK6,North Luzon 2,Rachelle Belmonte,MemoXpress
5,Memoxpress Robinsons Place Santiago,Robinsons Santiago,For Opening,2024-05-16,MZ0GKP9,North Luzon 2,Rachelle Belmonte,MemoXpress
6,Memoxpress Vista Mall Bataan,Vista Mall Bataan,For Opening,2024-05-16,MHAIZ42,North Luzon 2,Rachelle Belmonte,MemoXpress
7,Memoxpress Harbor Point Olongapo,Harbor Point Mall Subic,For Opening,2024-05-16,M85NL6S,North Luzon 2,Rachelle Belmonte,MemoXpress
8,Jr Mx Iba Town Center Zambales,Iba Town Center,For Opening,2024-05-16,MCRDDLY,North Luzon 2,Rachelle Belmonte,MemoXpress
9,Jr Mx Waltermart Subic,Waltermart Subic,For Opening,2024-05-16,M9P4UAF,North Luzon 2,Rachelle Belmonte,MemoXpress


In [3]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [4]:
df['BrandStores'].value_counts()

BrandStores
MemoXpress    17
Name: count, dtype: int64

In [5]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MK5LZTE    1
M9P4UAF    1
MHCRP9G    1
MIL9V6F    1
M2RP8DA    1
MKGSR62    1
M6LBA3A    1
MLAYROX    1
MCRDDLY    1
MORXEFK    1
M85NL6S    1
MHAIZ42    1
MZ0GKP9    1
MZPBVK6    1
MSVRCTU    1
MQDX72I    1
MEQ7VG1    1
Name: count, dtype: int64

In [6]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,MemoXpress,Jr Mx Centris Eton,Eton Centris Mall,For Opening,2024-05-16,MK5LZTE,NCR North,Sander Maghirang
1,MemoXpress,Jr Mx Mplace South Triangle Quezon City,Standalone,For Opening,2024-05-16,MORXEFK,NCR North,Sander Maghirang
2,MemoXpress,Jr Mx Sun Residences Quezon City,Standalone,For Opening,2024-05-16,MQDX72I,NCR North,Sander Maghirang
3,MemoXpress,Jr Mx Xentro Mall Vigan,Xentro Mall Vigan,For Opening,2024-05-16,MSVRCTU,North Luzon 2,Rachelle Belmonte
4,MemoXpress,Mx Robinsons Tuguegarao,Robinsons Tuguegarao,For Opening,2024-05-16,MZPBVK6,North Luzon 2,Rachelle Belmonte
5,MemoXpress,Memoxpress Robinsons Place Santiago,Robinsons Santiago,For Opening,2024-05-16,MZ0GKP9,North Luzon 2,Rachelle Belmonte
6,MemoXpress,Memoxpress Vista Mall Bataan,Vista Mall Bataan,For Opening,2024-05-16,MHAIZ42,North Luzon 2,Rachelle Belmonte
7,MemoXpress,Memoxpress Harbor Point Olongapo,Harbor Point Mall Subic,For Opening,2024-05-16,M85NL6S,North Luzon 2,Rachelle Belmonte
8,MemoXpress,Jr Mx Iba Town Center Zambales,Iba Town Center,For Opening,2024-05-16,MCRDDLY,North Luzon 2,Rachelle Belmonte
9,MemoXpress,Jr Mx Waltermart Subic,Waltermart Subic,For Opening,2024-05-16,M9P4UAF,North Luzon 2,Rachelle Belmonte


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BrandStores      17 non-null     object        
 1   POS NAME         17 non-null     object        
 2   STORE LOCATION   17 non-null     object        
 3   STATUS           17 non-null     object        
 4   DATE ACTIVATION  17 non-null     datetime64[ns]
 5   PURPLE KEY       17 non-null     object        
 6   REGION           17 non-null     object        
 7   TSM              17 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 1.2+ KB


In [8]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [9]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

In [10]:
print(df['DateOpened'].dtype)
# datetime64[ns]

datetime64[ns]


In [11]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-05-16 00:00:00


In [12]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    17 non-null     object
 1   POS            17 non-null     object
 2   StoreLocation  17 non-null     object
 3   Status         17 non-null     object
 4   DateOpened     17 non-null     object
 5   PurpleKey      17 non-null     object
 6   Region         17 non-null     object
 7   TSM            17 non-null     object
dtypes: object(8)
memory usage: 1.2+ KB


In [13]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    17 non-null     object        
 1   POS            17 non-null     object        
 2   StoreLocation  17 non-null     object        
 3   Status         17 non-null     object        
 4   DateOpened     17 non-null     datetime64[ns]
 5   PurpleKey      17 non-null     object        
 6   Region         17 non-null     object        
 7   TSM            17 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 1.2+ KB


In [15]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,MemoXpress,Jr Mx Centris Eton,Eton Centris Mall,For Opening,2024-05-16,MK5LZTE,NCR North,Sander Maghirang
1,MemoXpress,Jr Mx Mplace South Triangle Quezon City,Standalone,For Opening,2024-05-16,MORXEFK,NCR North,Sander Maghirang
2,MemoXpress,Jr Mx Sun Residences Quezon City,Standalone,For Opening,2024-05-16,MQDX72I,NCR North,Sander Maghirang
3,MemoXpress,Jr Mx Xentro Mall Vigan,Xentro Mall Vigan,For Opening,2024-05-16,MSVRCTU,North Luzon 2,Rachelle Belmonte
4,MemoXpress,Mx Robinsons Tuguegarao,Robinsons Tuguegarao,For Opening,2024-05-16,MZPBVK6,North Luzon 2,Rachelle Belmonte


In [16]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [17]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f901b0b8-039d-47fc-b203-47dc66007444>

In [18]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID c03c7669-560a-4489-85e8-0707d9642c3d successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(597, 8)


In [19]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID bad0ddf1-0b75-4d68-b3ac-9c536835fd89 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [20]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Hardware Sugar        2
Home Along           90
Home Factory          2
Homeworks            13
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           37
PHPS                  2
Power Mac Center      5
Rulls               154
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               22
Name: count, dtype: int64

In [21]:
dfsil.shape

(597, 8)

In [22]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 614 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [23]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 55c84041-fabf-48d7-b5eb-0a25a9f974dd successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (614, 8)


In [24]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
604,MemoXpress,Memoxpress Harbor Point Olongapo,Harbor Point Mall Subic,For Opening,2024-05-16,M85NL6S,North Luzon 2,Rachelle Belmonte
605,MemoXpress,Jr Mx Iba Town Center Zambales,Iba Town Center,For Opening,2024-05-16,MCRDDLY,North Luzon 2,Rachelle Belmonte
606,MemoXpress,Jr Mx Waltermart Subic,Waltermart Subic,For Opening,2024-05-16,M9P4UAF,North Luzon 2,Rachelle Belmonte
607,MemoXpress,Jr Mx Waltermart Balanga Bataan,Waltermart Balanga,For Opening,2024-05-16,MLAYROX,North Luzon 2,Rachelle Belmonte
608,MemoXpress,Jr Mx Pavilion Mall Laguna,Pavillion Mall Laguna,For Opening,2024-05-16,M6LBA3A,South Luzon 2,Kenneth Mendoza
609,MemoXpress,Memoxpress Paseo De Sta Rosa,Standalone,For Opening,2024-05-16,MEQ7VG1,South Luzon 2,Kenneth Mendoza
610,MemoXpress,Memoxpress Ayala Legaspi,Ayala Mall Albay,For Opening,2024-05-16,MKGSR62,South Luzon 3,Jan jan Dela Cruz
611,MemoXpress,Mx Robinsons Place Naga,Robinsons Naga,For Opening,2024-05-16,M2RP8DA,South Luzon 3,Jan jan Dela Cruz
612,MemoXpress,Jr Mx Waltermart Lotus Mall,Waltermart Imus,For Opening,2024-05-16,MIL9V6F,South Luzon 1 (B),Ricky Sarayno
613,MemoXpress,Memoxpress The District Imus,The District Mall Imus,For Opening,2024-05-16,MHCRP9G,South Luzon 1 (B),Ricky Sarayno


In [25]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Hardware Sugar        2
Home Along           90
Home Factory          2
Homeworks            13
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Rulls               154
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               22
Name: count, dtype: int64

In [26]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,2,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,3,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
12,556,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
13,559,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
14,566,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
15,567,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
4,482,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
5,483,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
7,494,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
8,495,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [27]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [28]:
df[df['PurpleKey']== 'MD0APY8']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
212,Junliz,Mobistar Metro Plaza Caloocan,Metro Plaza Caloocan,For Opening,2024-05-11,MD0APY8,NCR North,Sander Maghirang


In [29]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=37657dc9-8b39-49ed-9e29-ecec80efeb29>

In [30]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3c152995-ee43-47f6-ace1-634560e078ed>

In [31]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=30e98c09-a779-4ecb-abeb-f81a3964a4d6>

In [32]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 96e6dea0-f732-4118-8ba5-147594d93463 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(614, 8)

In [33]:
df[df['BrandStores']=='Junliz'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
178,Junliz,Junliz Nova Plaza Mall,Nova Plaza Mall,For Opening,2024-03-06,MP6R2OC,NCR North,Sander Maghirang
179,Junliz,Junliz Paso De Blas Valenzuela,Standalone,For Opening,2024-03-06,MODES01,NCR North,Sander Maghirang
214,Junliz,Junliz Expo,Standalone,POS Operational,2024-05-01,MPBE087,NCR North,Sander Maghirang
215,Junliz,Mobistar Metro Plaza Novaliches,Metro Plaza Nova,For Opening,2024-05-11,M1OGRVH,NCR North,Sander Maghirang
216,Junliz,Mobistar Zabarte,Zabarte Mall,For Opening,2024-05-11,MTXPCFL,NCR North,Sander Maghirang
217,Junliz,Mobistar Robinsons Novaliches,Robinsons Novaliches,For Opening,2024-05-11,MEC3R0S,NCR North,Sander Maghirang
219,Junliz,Junliz General Luis,Standalone,For Opening,2024-05-11,MJ1VWQ3,NCR North,Sander Maghirang
220,Junliz,Mobistar Metro Plaza Caloocan,Metro Plaza Caloocan,For Opening,2024-05-11,MD0APY8,NCR North,Sander Maghirang


In [34]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
3,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
4,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
5,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
6,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
7,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
8,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
9,MemoXpress,Memoxpress Sta Lucia East Grand Mall,Sta Lucia Mall,For Opening,2024-04-15,MRTAIQG,NCR East,Albee David


In [35]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
388,MemoXpress,Jr Mx Xentro Mall Vigan,Xentro Mall Vigan,For Opening,2024-05-16,MSVRCTU,North Luzon 2,Rachelle Belmonte
383,MemoXpress,Jr Mx Waltermart Balanga Bataan,Waltermart Balanga,For Opening,2024-05-16,MLAYROX,North Luzon 2,Rachelle Belmonte
390,MemoXpress,Mx Robinsons Tuguegarao,Robinsons Tuguegarao,For Opening,2024-05-16,MZPBVK6,North Luzon 2,Rachelle Belmonte
389,MemoXpress,Memoxpress Vista Mall Bataan,Vista Mall Bataan,For Opening,2024-05-16,MHAIZ42,North Luzon 2,Rachelle Belmonte
387,MemoXpress,Jr Mx Waltermart Subic,Waltermart Subic,For Opening,2024-05-16,M9P4UAF,North Luzon 2,Rachelle Belmonte
386,MemoXpress,Memoxpress Harbor Point Olongapo,Harbor Point Mall Subic,For Opening,2024-05-16,M85NL6S,North Luzon 2,Rachelle Belmonte
114,MemoXpress,Jr Mx Waltermart Lotus Mall,Waltermart Imus,For Opening,2024-05-16,MIL9V6F,South Luzon 1 (B),Ricky Sarayno
115,MemoXpress,Memoxpress The District Imus,The District Mall Imus,For Opening,2024-05-16,MHCRP9G,South Luzon 1 (B),Ricky Sarayno
223,MemoXpress,Jr Mx Sun Residences Quezon City,Standalone,For Opening,2024-05-16,MQDX72I,NCR North,Sander Maghirang
385,MemoXpress,Memoxpress Robinsons Place Santiago,Robinsons Santiago,For Opening,2024-05-16,MZ0GKP9,North Luzon 2,Rachelle Belmonte


In [36]:
df[df['PurpleKey'] =='M3Z10RP']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
495,Ansons,Ansons Greenhills,Greenhills,For Opening,2024-04-15,M3Z10RP,NCR South 1,Klein Justine Tamayo


In [37]:
df[df['BrandStores']=='ESLA']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
30,ESLA,ESLA Malabon,Standalone,For Opening,2024-04-22,M56ROOZ,NCR East,Albee David
31,ESLA,ESLA Balanga 2,Standalone,For Opening,2024-04-22,MLANGU7,NCR East,Albee David
47,ESLA,ESLA Montalban 2,Standalone,For Opening,2024-02-15,M9Q3NCO,NCR East,Charm Moltio
48,ESLA,ESLA Pasig 3,Standalone,For Opening,2024-02-15,MBS5T2W,NCR East,Charm Moltio
49,ESLA,ESLA Antipolo,Standalone,For Opening,2024-02-15,MDT8FNQ,NCR East,Charm Moltio
50,ESLA,ESLA San Mateo 1,Standalone,For Opening,2024-02-15,MD6S28C,NCR East,Charm Moltio
51,ESLA,ESLA Binangonan Bayan,Standalone,For Opening,2024-02-15,MAVILNK,NCR East,Charm Moltio
52,ESLA,ESLA Cogeo,Standalone,For Opening,2024-02-15,M9N5R2P,NCR East,Charm Moltio
53,ESLA,ESLA Taytay,Standalone,For Opening,2024-02-15,MWI9HXN,NCR East,Charm Moltio
54,ESLA,ESLA Marikina 1,Standalone,For Opening,2024-02-15,M06W4RG,NCR East,Charm Moltio
